In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.momentum import ROCIndicator
from ta.trend import AroonIndicator
from ta.trend import SMAIndicator
from ta.trend import DPOIndicator
from ta.trend import MACD
from ta.trend import EMAIndicator
from ta.volume import EaseOfMovementIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import OnBalanceVolumeIndicator
from ta.volatility import BollingerBands
from ta.volatility import KeltnerChannel
from ta.volatility import AverageTrueRange
from ta.volatility import DonchianChannel
#from ta.others import DailyLogReturnIndicator
from ta.momentum import ROCIndicator
import time
from IPython.display import clear_output
from datetime import datetime

start=(datetime.now())

sip500=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].iloc[:,0])
sip400=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')[0].iloc[:,1])
sip600=np.array(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')[1].iloc[:,1])
payload=np.append(sip500,sip400)
payload=np.append(payload,sip600)
payload=np.unique(payload)
payload=payload[payload!='BRK.B']
payload=payload[payload!='JW.A']
payload=payload[payload!='BF.B']

#print(payload)


class algo:
    start="2010-01-04"
    end='2021-12-04'
    interval='1wk'

    dff=pd.DataFrame()
    
    def __init__(self,stock):
        self.stock=stock
        
    # get raw stock data       
    def getdata(self):
        try:
            df=(yf.download(self.stock,algo.start,algo.end,interval=algo.interval).dropna())

            #df['log_current']=np.log(df.Close/df.Open)
            df['log_current']=((df.Close-df.Open)/df.Open)
            df['stock']=self.stock
            df['Year'] = df.index.strftime('%Y-%m-%d').str[:4]

            #output = stock data from start to end
            clear_output(wait=False)
        except:
            pass
    
        return(df)
    
    
    def stockloop():
        l1=0
        
        l2=len(payload)
        dff=algo.dff
        boom=pd.DataFrame()
        for i in range (l1,l2):
            try:
                rawdata=algo(payload[i]).getdata()
                dff=pd.concat([dff,rawdata])
                print(i)
            except Exception as e:
                print(e)
                pass
        dff.set_index([dff.stock,dff.index],inplace=True)
        return(dff)

dff=algo.stockloop()
#dff.to_csv('raw.csv')
uniq=(dff.index.unique(level=0))

finish = datetime.now()
print('Duration: {}'.format(finish - start))
#time.sleep(600)

1500
Duration: 0:15:36.985993


In [10]:
start=(datetime.now())
#print(dff)
class algo2:
    dfmass=pd.DataFrame()
    thr1=1
    thr2=99
    end=['2022-01-01','2021-01-01','2020-01-01','2019-01-01','2018-01-01','2017-01-01','2016-01-01','2015-01-01','2014-01-01']
    def __init__(self,stock):
        self.stock=stock

    def indicators(self):
        df=self.stock
        #df['ema5']=EMAIndicator(close=df.Close.shift(1),window=5).ema_indicator()
        #df['ema10']=EMAIndicator(close=df.Close.shift(1),window=10).ema_indicator()
        #df['ema20']=EMAIndicator(close=df.Close.shift(1),window=20).ema_indicator()
        #df['ema10_20']=df.ema10/df.ema20
        ##df['log_past']=DailyLogReturnIndicator(close=df.Close.shift(1)).daily_log_return()
        #df['ema1']=((df.Close-df.Close.shift(1))/df.Close.shift(1)).shift(1)
        #df['condition']=df.ema1
        df['DR']=((df.Close-df.Close.shift(1))/df.Close.shift(1)).shift(1)
       # df['condition']=EMAIndicator(close=df.Close.shift(1),window=5).ema_indicator()
        #df['obv']=OnBalanceVolumeIndicator(close=df.Close.shift(1),volume=df.Volume.shift(1)).on_balance_volume()
       # df['rsi']=RSIIndicator(close=df.Close.shift(1),window=12).rsi()
        df['FI']=ForceIndexIndicator(close=df.Close.shift(1),volume=df.Volume.shift(1),window=12).force_index()
       # df['Aroon']=AroonIndicator(close=df.Close.shift(1),window=12).aroon_indicator()
        df['MACD']=MACD(close=df.Close.shift(1)).macd()
        df['condition1']=df.DR
        df['condition2']=df.FI
        df['condition3']=df.MACD
        #df['roc']=ROCIndicator(close=df.Close.shift(1)).roc()
        df=df.dropna()

        #output = stock data with indicators
        return(df)

    def percentiles(self):
        df=self.stock
        dic11={}
        dic12={}
        dic21={}
        dic22={}
        dic31={}
        dic32={}
        for t in range (1,len(algo2.end)):
            try:
                time=algo2.end[t]
                df=(df[df.index<time])
  
                dic11[time]=np.nanpercentile((df.condition1),1)
                #dic11[time]=20
                dic12[time]=np.nanpercentile((df.condition1),99)
                dic21[time]=np.nanpercentile((df.condition2),50)
                #dic22[time]=np.nanpercentile((df.condition2),50)
                dic31[time]=np.nanpercentile((df.condition3),50)
                #dic32[time]=np.nanpercentile((df.condition3),50)
                
            except:
                pass
        #output = 2 dictionaries: year - percentile of preveious year (1,99)
        diczip=(dic11,dic12,dic21,dic31)
        return(diczip)

        return(dict21,dict22)
    def trade(self,percentiles):
        df=self.stock
        dff=pd.DataFrame()
        for t in range (1,len(algo2.end)):
            origin=algo2.end[t]
            end=algo2.end[t-1]
            #print(orgin,end)
            #select origin<stock data<end
            dft=(df[(df.index>=origin) & (df.index <end)])
            
            #apply trade conditions based on thresholds
            dft['trade_pass']=np.select(
                
            [(dft.condition1<percentiles[0][origin])
             &(dft.condition2<percentiles[2][origin])
             &(dft.condition3<percentiles[3][origin])
             ,
             
             (dft.condition1>percentiles[1][origin])
             &(dft.condition2>percentiles[2][origin])
             &(dft.condition3>percentiles[3][origin])]
                                                                 
            ,[1,-1],default=0)
            
            
            
            
            #dft['Bottom_Percentile']=perc1[origin]
            #dft['Top_Percentile']=perc2[origin]
            dff=pd.concat([dff,dft])
        return(dff)
    def stockloop():
        boom=pd.DataFrame()
        for i in uniq:

            rawdata=(dff.loc[i])
            ta=algo2(rawdata).indicators()
            percentiles=algo2(ta).percentiles()
            trades=algo2(ta).trade(percentiles)
            boom=pd.concat([boom,trades])

        return(boom)


boom=algo2.stockloop()
#print(boom)
boom.to_csv('all_data.csv')

finish = datetime.now()
print('Duration: {}'.format(finish - start))


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Duration: 0:02:47.342078


In [11]:
def yearloop(boom):

    years=np.array(['2021','2020','2019','2018','2017','2016','2015','2014'])
   
    long=boom[boom['trade_pass']==1]
    short=boom[boom['trade_pass']==-1]        
    short['log_current']=-short['log_current']+1
    long['log_current']=long['log_current']+1
    group=([long,short])
    long.to_csv('all_long.csv')
    short.to_csv('all_short.csv')

    #print(short)
    temp=np.array([])
    for g in group: 
     
        for y in years:

            gr=g[g.Year==y]
    
            trade_count=gr.shape[0]
            try:
                win_rate=(((gr[gr['log_current']>1]).shape[0]))/trade_count
            except:
                win_rate=0
            cumilative=np.prod(gr.groupby(level=0).mean().log_current)-1
            minimum=np.min(gr.groupby(level=0).mean().log_current)-1
            temp1=np.array([y,trade_count,win_rate,cumilative,minimum])
            temp=np.append(temp,temp1)

    return(temp)

daf=(yearloop(boom).reshape(16,5))
dper=pd.DataFrame(daf)
dper['stock_count']=np.count_nonzero(uniq)
print(dper.round(1))
dper.to_csv('donut.csv')

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


       0     1                    2                     3  \
0   2021   137   0.5547445255474452   0.27256384234551145   
1   2020  4847   0.6562822364349082  0.024819405155138474   
2   2019   541   0.5360443622920518   0.03323702544016682   
3   2018  1200   0.5833333333333334    0.2816201450678051   
4   2017   367   0.5013623978201635     0.384690597029953   
5   2016   878  0.45899772209567197   0.49768516437250354   
6   2015   762    0.583989501312336    0.3847131386696203   
7   2014   376   0.5930851063829787    0.5220730801324345   
8   2021   599   0.5058430717863105  -0.04461708072890913   
9   2020  1106    0.566003616636528   -0.5533322075755112   
10  2019   380  0.49473684210526314   0.06691283611076715   
11  2018   486   0.5555555555555556    0.5133471323791476   
12  2017   278   0.4856115107913669  -0.16597831509930316   
13  2016   761  0.43495400788436267   -0.2576412653660092   
14  2015   388   0.5541237113402062   0.26726342677054893   
15  2014   318   0.49371